In [3]:
import redis
import msgpack
import msgpack_numpy as mnp
pool = redis.ConnectionPool(host='34.69.39.105', port=6379, db=0)
r = redis.Redis(connection_pool=pool)

In [4]:
import numpy as np
from bokeh.models import Circle, ColumnDataSource,Plot,ColorBar, HoverTool
from bokeh.plotting import figure, save
from bokeh.transform import linear_cmap
from bokeh.palettes import YlOrBr
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
import geopandas as gpd

In [5]:
def merc(lon, lat):
    ''' Get mercator projection co-ordinates from latitude and logitude'''
    x=[]
    y=[]
    r_major = 6378137.000
    for i in range(len(lon)):
        x.append(r_major * np.radians(lon[i])) 
        scale = x[i]/lon[i]
        y.append(180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
         lat[i]* (np.pi/180.0)/2.0)) * scale)
    return x, y

In [54]:
# Get data from redis server
predmeans = mnp.unpackb(r.get('predmeans'), raw=True)
predvars = mnp.unpackb(r.get('predvars'), raw=True)
Xtest = mnp.unpackb(r.get('Xtest'), raw=True)
Xtraining = mnp.unpackb(r.get('Xtraining'), raw=True)

In [55]:
# Get latitude and longitude from Xtest and convert to mercator
lon = []
lat = []
for i in range(len(Xtest[0][1])):
    lon.extend(Xtest[0][i].tolist())
    lat.extend(Xtest[1][i].tolist())
x, y = merc(lon,lat)

In [56]:
# Create array of sizes for markers from uncertainty predictions
sizes = 15*np.log(1 + 300/np.sqrt(predvars))
min(sizes)

array([48.05711462])

In [61]:
len(x_filtered)

22500

In [165]:
# Get map tiles
tile_provider = get_provider(CARTODBPOSITRON)

# Create ColumnDataSource from coordinates, sizes and predicted means
source = ColumnDataSource(dict(x=x, y=y, sizes=sizes, means=predmeans))

# Create linear colour map for pollution data
mapper = linear_cmap(field_name='means', palette=YlOrBr[9][::-1] ,low=0 ,high=max(predmeans)[0])

# Plot figure and add tiles 
mercator = figure(title=None, x_range=(min(x), max(x)), y_range=(min(y), max(y)),
           x_axis_type="mercator", y_axis_type="mercator")
mercator.add_tile(tile_provider)

# Create and plot glyphs for predictions
glyph = Circle(x="x", y="y", radius="sizes", line_color=mapper, fill_color=mapper, fill_alpha=.8, line_width=0)
mercator.add_glyph(source, glyph)

# Add colour bar 
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
mercator.add_layout(color_bar, 'right')

outfp = r"output/kampala_circle.html"

# Save the plot by passing the plot -object and output path
save(obj=mercator, filename=outfp)

FileNotFoundError: [Errno 2] No such file or directory: 'output/kampala_circle.html'

In [62]:
filter = []
for var in predvars:
    if np.sqrt(var) > np.sqrt(predvars.mean()):
        filter.append(False)
    else:
        filter.append(True)

In [63]:
x_filtered = np.asarray(x)[filter]
y_filtered = np.asarray(y)[filter]
mean_filtered = predmeans[filter]
vars_filtered = predvars[filter]
sizes_filtered = 15*np.log(1 + 300/np.sqrt(vars_filtered))
#alphas = (0.2 + 0.8*min(vars_filtered)/vars_filtered)
alphas = (1/np.log(1 + np.sqrt(vars_filtered)))/max((1/np.log(1 + np.sqrt(vars_filtered))))
lon_filtered = np.asarray(lon)[filter]
lat_filtered = np.asarray(lat)[filter]

In [64]:
# Get map tiles
tile_provider = get_provider(CARTODBPOSITRON)

# Create ColumnDataSource from coordinates, sizes and predicted means
source1 = ColumnDataSource(dict(x=x_filtered, y=y_filtered, sizes=sizes_filtered, means=mean_filtered, 
                                vars=np.sqrt(vars_filtered), alphas=alphas, lat=lat_filtered, lon=lon_filtered))

# Create linear colour map for pollution data
mapper1 = linear_cmap(field_name='means', palette=YlOrBr[9][::-1] ,low=0 ,high=max(mean_filtered)[0])

# Tooltips
tooltips = [("Lat/long", "@lat, @lon"),
                ("Predicted PM2.5", "@means"),
                ("Predicted Variance", "@vars")]

# Plot figure and add tiles 
mercator_filtered = figure(title=None, x_range=(min(x), max(x)), y_range=(min(y), max(y)),
           x_axis_type="mercator", y_axis_type="mercator", tooltips=tooltips, tools = "pan,wheel_zoom,box_zoom,reset,hover")
mercator_filtered.add_tile(tile_provider)

# Create and plot glyphs for predictions
glyph1 = Circle(x="x", y="y", radius=60, line_color="white", fill_color=mapper1, fill_alpha="alphas", line_width=1, 
                line_alpha=1)
mercator_filtered.add_glyph(source1, glyph1)

# Add colour bar 
color_bar = ColorBar(color_mapper=mapper1['transform'], width=8,  location=(0,0))
mercator_filtered.add_layout(color_bar, 'right')

outfp = r"data/kampala_circle_filtered.html"

# Save the plot by passing the plot -object and output path
save(obj=mercator_filtered, filename=outfp)

'C:\\Projects\\AirQo\\src\\Maps\\data\\kampala_circle_filtered.html'

In [137]:
import json
from bokeh.embed import json_item
item_text = json.dumps(json_item(mercator_filtered, "myplot"))

In [199]:
max((1/np.log(1 + np.sqrt(predvars)))/max((1/np.log(1 + np.sqrt(predvars)))))

array([1.])

In [215]:
from bokeh.events import Event, LODEnd
mercator_filtered.on_event(LODEnd)

In [84]:
from scipy import interpolate
f = interpolate.RectBivariateSpline(gridx, gridy, predmeans.reshape(150,150))

In [85]:
gridx, gridy = merc(Xtest[0][0].tolist(), Xtest[1].T[0])

In [86]:
xnew = np.linspace(mercator_filtered.x_range.start, mercator_filtered.x_range.end, 20)
ynew = np.linspace(mercator_filtered.y_range.start, mercator_filtered.y_range.end, 20)

In [82]:
f(xnew,ynew).reshape(400).shape

(400,)

In [83]:
xnew.shape

(20,)